<a href="https://colab.research.google.com/github/arshiii08/MNIST-GAN-Implementation/blob/main/MNIST_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, Reshape, Flatten, Input
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Dropout
from tensorflow.keras.optimizers import Adam
import os

In [ ]:
# Set random seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
# Create directory for saving images
if not os.path.exists("gan_images"):
    os.makedirs("gan_images")

In [ ]:
# Step 1: Load and preprocess the MNIST dataset
print("Loading MNIST dataset...")
(X_train, _), (_, _) = mnist.load_data()
X_train = X_train / 127.5 - 1.0  # Normalize to [-1, 1]
X_train = np.expand_dims(X_train, axis=3)  # Add channel dimension
print(f"Dataset loaded. Shape: {X_train.shape}")

Loading MNIST dataset...
11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Dataset loaded. Shape: (60000, 28, 28, 1)


In [ ]:
# Step 2: Set up model parameters
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

z_dim = 100  # Dimension of the latent space
learning_rate = 0.0002
batch_size = 128
epochs = 2000  # Increased for better results
sample_interval = 200  # Save images every 200 epochs

In [ ]:
# Step 3: Build the improved Generator model with convolutional layers
def build_generator(z_dim):
    model = Sequential()

    # Foundation for 7x7 image
    model.add(Dense(7 * 7 * 256, input_dim=z_dim))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 256)))

    # Upsample to 14x14
    model.add(Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    # Upsample to 28x28
    model.add(Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha=0.2))

    # Output layer
    model.add(Conv2D(1, (5, 5), padding='same', activation='tanh'))

    print("Generator:")
    model.summary()

    noise = Input(shape=(z_dim,))
    img = model(noise)

    return Model(noise, img)


In [ ]:
# Step 4: Build the improved Discriminator model with convolutional layers
def build_discriminator():
    model = Sequential()

    # First convolutional layer
    model.add(Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=img_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))

    # Second convolutional layer
    model.add(Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.3))

    # Flatten and dense layer
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))

    print("Discriminator:")
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

# Function to generate and save images
def generate_and_save_images(generator, epoch, z_dim, n_samples=25, rows=5, cols=5):
    noise = np.random.normal(0, 1, (n_samples, z_dim))
    gen_imgs = generator.predict(noise)

    # Rescale images from [-1, 1] to [0, 1]
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig = plt.figure(figsize=(cols*2, rows*2))

    for i in range(n_samples):
        plt.subplot(rows, cols, i+1)
        plt.imshow(gen_imgs[i, :, :, 0], cmap='gray')
        plt.axis('off')

    plt.tight_layout()
    plt.savefig(f'gan_images/epoch_{epoch}.png')
    plt.close()
    print(f"Saved images for epoch {epoch}")

In [ ]:
# Step 5: Build and compile the models
print("Building models...")
generator = build_generator(z_dim)
discriminator = build_discriminator()

# For the combined model, we only train the generator
discriminator.trainable = False

# The generator takes noise as input and generates images
z = Input(shape=(z_dim,))
img = generator(z)

# The discriminator determines validity of the generated images
validity = discriminator(img)

# The combined model (stacked generator and discriminator)
gan = Model(z, validity)
gan.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=learning_rate, beta_1=0.5))

# Compile the discriminator separately
discriminator.compile(loss='binary_crossentropy',
                      optimizer=Adam(learning_rate=learning_rate, beta_1=0.5),
                      metrics=['accuracy'])

print("Models built and compiled!")

Building models...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Generator:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "sequential"

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)                    │ (None, 14, 14, 64)          │           1,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 14, 14, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 7, 7, 128)           │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 7, 7, 128)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 6272)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │           6,273 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 212,865 (831.50 KB)

 Trainable params: 212,865 (831.50 KB)

 Non-trainable params: 0 (0.00 B)

Models built and compiled!


In [ ]:
# Step 6: Training function
def train_gan(epochs, batch_size, sample_interval):
    # Labels for real and fake images with label smoothing
    valid = np.ones((batch_size, 1)) * 0.9  # Label smoothing
    fake = np.zeros((batch_size, 1)) + 0.1  # Label smoothing

    # Save initial random generations (before any training)
    generate_and_save_images(generator, 0, z_dim)

    for epoch in range(epochs):
        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random batch of real images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        real_imgs = X_train[idx]

        # Add small random noise to images (helps with training stability)
        real_imgs = real_imgs + 0.05 * np.random.normal(0, 1, real_imgs.shape)
        real_imgs = np.clip(real_imgs, -1, 1)  # Ensure values stay in [-1, 1]

        # Generate a batch of fake images
        noise = np.random.normal(0, 1, (batch_size, z_dim))
        gen_imgs = generator.predict(noise)

        # Train the discriminator
        d_loss_real = discriminator.train_on_batch(real_imgs, valid)
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        # Train the generator (to have the discriminator label samples as valid)
        noise = np.random.normal(0, 1, (batch_size, z_dim))
        g_loss = gan.train_on_batch(noise, valid)

        # Print progress
        if epoch % sample_interval == 0:
            print(f"Epoch {epoch}/{epochs} [D loss: {d_loss[0]:.4f}, acc.: {100*d_loss[1]:.2f}%] [G loss: {g_loss:.4f}]")

            # Generate and save images
            generate_and_save_images(generator, epoch, z_dim)

    # Generate final set of images after training completes
    generate_and_save_images(generator, epochs, z_dim)
    print("Training completed!")

    # Save the trained models
    generator.save('mnist_generator.h5')
    discriminator.save('mnist_discriminator.h5')
    print("Models saved!")

In [ ]:
# Step 7: Run the training
print("Starting GAN training...")
train_gan(epochs=epochs, batch_size=batch_size, sample_interval=sample_interval)

Starting GAN training...
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Saved images for epoch 0
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step  


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0/2000 [D loss: 0.6882, acc.: 0.00%] [G loss: 0.7137]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Saved images for epoch 0
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
4/4

Saved images for epoch 2000
Training completed!
Models saved!
